In [1]:

'''
Modified version:
Kiran Gunnam
A Convolutional Network implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/

'''

from __future__ import print_function

import tensorflow as tf
from tensorflow.python import debug as tf_debug

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('model_data/', one_hot=True)
print ("HI")
# Parameters
learning_rate = 0.001
training_iters = 50000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 1.0 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)


# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

def avgpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.avg_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

wt1 = weights['wc1']
wt2 = weights['wc2']
wd1 = weights['wd1']

# Construct model
pred = conv_net(x, weights, biases, keep_prob)
tf.summary.histogram("W1",wt1 )
tf.summary.histogram("W1",wt2 )
tf.summary.histogram("W1",wd1 )
# Define loss and optimizer
with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    tf.summary.scalar("cost",cost)
# Evaluate model
with tf.name_scope("accuracy"):

    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))*100
    tf.summary.scalar("accuracy",accuracy)
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    writer = tf.summary.FileWriter("./logs/cnn_logs2", sess.graph) # for 0.8
    merged = tf.summary.merge_all()
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        #if step % display_step == 0:
            # Calculate batch loss and accuracy
        s, loss, acc = sess.run([merged,cost, accuracy], feed_dict={x: batch_x,
                                                          y: batch_y,
                                                          keep_prob: 1.})
        print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
              "{:.6f}".format(loss) + ", Training Accuracy= " + \
              "{:.5f} %".format(acc))
        writer.add_summary(s, step)
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting model_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting model_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting model_data/t10k-images-idx3-ubyte.gz
Extracting model_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
HI
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Iter 128, Minibatch Loss= 35768.312500, Training Accuracy= 18.75000 %
Iter 256, Minibatch Loss= 30267.4257

Iter 10496, Minibatch Loss= 1537.576660, Training Accuracy= 86.71875 %
Iter 10624, Minibatch Loss= 2534.583984, Training Accuracy= 80.46875 %
Iter 10752, Minibatch Loss= 2169.167480, Training Accuracy= 86.71875 %
Iter 10880, Minibatch Loss= 1652.381348, Training Accuracy= 85.15625 %
Iter 11008, Minibatch Loss= 2079.388184, Training Accuracy= 85.93750 %
Iter 11136, Minibatch Loss= 1523.394897, Training Accuracy= 89.06250 %
Iter 11264, Minibatch Loss= 2289.893066, Training Accuracy= 80.46875 %
Iter 11392, Minibatch Loss= 2339.970215, Training Accuracy= 86.71875 %
Iter 11520, Minibatch Loss= 1496.336914, Training Accuracy= 88.28125 %
Iter 11648, Minibatch Loss= 2462.299805, Training Accuracy= 80.46875 %
Iter 11776, Minibatch Loss= 1911.314575, Training Accuracy= 87.50000 %
Iter 11904, Minibatch Loss= 2513.520996, Training Accuracy= 84.37500 %
Iter 12032, Minibatch Loss= 2185.581055, Training Accuracy= 81.25000 %
Iter 12160, Minibatch Loss= 958.216797, Training Accuracy= 88.28125 %
Iter 12

Iter 25344, Minibatch Loss= 1240.623901, Training Accuracy= 90.62500 %
Iter 25472, Minibatch Loss= 1232.274536, Training Accuracy= 84.37500 %
Iter 25600, Minibatch Loss= 1114.026367, Training Accuracy= 87.50000 %
Iter 25728, Minibatch Loss= 1725.227051, Training Accuracy= 84.37500 %
Iter 25856, Minibatch Loss= 1046.946167, Training Accuracy= 85.93750 %
Iter 25984, Minibatch Loss= 1548.967407, Training Accuracy= 85.93750 %
Iter 26112, Minibatch Loss= 742.365662, Training Accuracy= 92.18750 %
Iter 26240, Minibatch Loss= 1868.934692, Training Accuracy= 84.37500 %
Iter 26368, Minibatch Loss= 1603.966919, Training Accuracy= 86.71875 %
Iter 26496, Minibatch Loss= 1251.406372, Training Accuracy= 89.84375 %
Iter 26624, Minibatch Loss= 1730.447510, Training Accuracy= 86.71875 %
Iter 26752, Minibatch Loss= 1137.102417, Training Accuracy= 90.62500 %
Iter 26880, Minibatch Loss= 1243.958618, Training Accuracy= 86.71875 %
Iter 27008, Minibatch Loss= 1190.584839, Training Accuracy= 90.62500 %
Iter 27

Iter 40320, Minibatch Loss= 784.258545, Training Accuracy= 94.53125 %
Iter 40448, Minibatch Loss= 990.738647, Training Accuracy= 90.62500 %
Iter 40576, Minibatch Loss= 415.966370, Training Accuracy= 94.53125 %
Iter 40704, Minibatch Loss= 720.035828, Training Accuracy= 94.53125 %
Iter 40832, Minibatch Loss= 382.503235, Training Accuracy= 93.75000 %
Iter 40960, Minibatch Loss= 510.460205, Training Accuracy= 94.53125 %
Iter 41088, Minibatch Loss= 546.823730, Training Accuracy= 92.96875 %
Iter 41216, Minibatch Loss= 1117.423218, Training Accuracy= 90.62500 %
Iter 41344, Minibatch Loss= 582.747559, Training Accuracy= 92.18750 %
Iter 41472, Minibatch Loss= 1276.516479, Training Accuracy= 91.40625 %
Iter 41600, Minibatch Loss= 542.097107, Training Accuracy= 93.75000 %
Iter 41728, Minibatch Loss= 660.060242, Training Accuracy= 92.96875 %
Iter 41856, Minibatch Loss= 665.591980, Training Accuracy= 93.75000 %
Iter 41984, Minibatch Loss= 842.329346, Training Accuracy= 91.40625 %
Iter 42112, Miniba